In [36]:
import numpy as np
from scipy.constants import c, m_p, e
from matplotlib import pyplot as plt

# Convert energy to time 

In [37]:
def synchrotron_momentum(max_E, time):
    mpeV = m_p * c**2 / e           # Proton mass in eV
    R0 = 26                         # Mean machine radius
    n_dip = 10                      # Number of dipoles
    dip_l = 4.4                     # Dipole length
    
    dip_angle = 2 * np.pi / n_dip   # Dipole bending angle
    rho = dip_l / dip_angle         # Dipole radius of curvature
    omega = 2 * np.pi * 50   
    
    Ek = np.array([70, max_E]) * 1e6 # Injection and extraction kinetic energies 
    E = Ek + mpeV                    # Injection and extraction kinetic energies
    p = np.sqrt(E**2 - mpeV**2)      # Injection and extraction momenta

    B = p / c / rho                  # Ideal magnetic field at injection and extraction energies
    
    Bdip = lambda t: (B[1] + B[0] - (B[1] - B[0]) * np.cos(omega * t)) / 2  # Idealised B-field variation with AC
    pdip = lambda t: Bdip(t) * rho * c                                      # Momentum from B-field in MeV
    
    return pdip(time*1E-3)

def synchrotron_kinetic_energy(max_E, time):
    mpeV = m_p * c**2 / e           # Proton mass in eV    
    # Relativistic Kinetic Energy = Relativistic Energy - mass
    return (np.sqrt(synchrotron_momentum(max_E, time)**2 + mpeV**2) - mpeV) # Return array in eV
    #return (np.sqrt(synchrotron_momentum(max_E, time)**2 + mpeV**2) - mpeV)/1E6 # Return array in MeV


In [39]:
x_data = np.linspace(-55, 800, data_points)
def divided_diff(x, y):
    '''
    function to calculate the divided
    differences table
    '''
    n = len(y)
    coef = np.zeros([n, n])
    # the first column is y
    coef[:,0] = y
    
    for j in range(1,n):
        for i in range(n-j):
            coef[i][j] = \
           (coef[i+1][j-1] - coef[i][j-1]) / (x[i+j]-x[i])
            
    return coef

def newton_poly(coef, x_data, x):
    '''
    evaluate the newton polynomial 
    at x
    '''
    n = len(x_data) - 1 
    p = coef[n]
    for k in range(1,n+1):
        p = coef[n-k] + (x -x_data[n-k])*p
    return p

def calibration_curve_beta(data_points):   
    x = np.array([70, 172, 374, 617, 780])
    y = np.array([2.22E-13, 2.59E-13, 4.31E-12, 1.60E-11, 3.50E-11])

    
    a_s = divided_diff(x, y)[0, :]
    x_new = np.linspace(0, 800, data_points) 
    return newton_poly(a_s, x, x_new)


In [12]:
x = np.array([70, 172, 374, 617, 780])
y = np.array([2.22E-13, 2.59E-13, 4.31E-12, 1.60E-11, 3.50E-11])

a_s = divided_diff(x, y)[0, :]

In [44]:
 
def get_calibration_curve(MaxE=800):
    """
    Get the calibration 
    """
    # get the interpolated calabration curve
    _data_points = 2366
    x_data = np.linspace(-55, 800, data_points)
    calibration_curve = calibration_curve_beta(len(x_data))
    calibration_curve = calibration_curve[366:]
    plt.plot(calibration_curve)
    # add the flat sections 
    start = np.full(100,calibration_curve[0])
    end = np.full(100,calibration_curve[-1])
    calibration_curve = np.array([*start,*calibration_curve, *end])
    return calibration_curve

